In [133]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from torch import nn,optim
import torch

import algo
import json

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
import numpy as np
import pandas as pd

import attack
import random




In [11]:
class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

class Train_args():

    def __init__(self, learning_rate, weight_decay, epoch):
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.epoch = epoch
         
def train_attack_model(model, train_data, train_target, train_args):

    optimizer = optim.Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)
    
    model.train()
    optimizer.zero_grad()
    output = model(train_data)
    loss = nn.CrossEntropyLoss()(output, train_target.to(torch.long))
    loss.backward(retain_graph=True)
    optimizer.step()

    return model

def attack_evaluation(model, x, y, dev="cpu", extended=False):

    model.eval()

    with torch.no_grad():
        output =  model(x)
        out_target = output.argmax(1, keepdim=True)
        correct = out_target.to(dev).eq(y.to(dev).view_as(out_target.to(dev))).sum().item()
        acc = correct/y.shape[0]

        predicted_positive = output.argmax(1, keepdim=True) == 1
        labeled_positive = y == 1
        tp = predicted_positive.to(dev) * labeled_positive.to(dev).view_as(out_target)
        tp_count = tp.to(dev).sum().item()
        
        if predicted_positive.to(dev).sum().item() != 0:
            pre = tp_count / predicted_positive.to(dev).sum().item()
        else:
            pre = 0
        if labeled_positive.to(dev).sum().item() !=0:
            rec = tp_count / labeled_positive.to(dev).sum().item()
        else:
            rec = 0
    if extended:
        predicted_negative = output.argmax(1, keepdim=True) == 0
        labeled_negative = y == 0
        tn = predicted_negative.to(dev) * labeled_negative.to(dev).view_as(out_target)
        tn_count = tn.to(dev).sum().item()

        fp_count = predicted_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        fn_count = labeled_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        
        return acc, pre, rec, tp_count, tn_count, fp_count, fn_count
    else:
        return acc, pre, rec

class model_params(object):

    def __init__(self, model_name):
        self.model_name      = model_name

In [12]:
# raw_data = pd.read_csv('../datasets/dataset_purchase', )
# y=raw_data['63']
# X=raw_data.drop('63', axis=1)
# y =  y.replace(100, 0)
# print(y.nunique())

# X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.1, random_state=42)
# print(X_train.shape, x_shadow.shape)

In [13]:
data = pd.read_csv('../DP-UTIL.nosync/loan_preprocessed.csv', index_col=0)


In [14]:
data.shape

(661283, 167)

In [110]:
#Loan
y = data['grade']
X = data.drop('grade', axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
tr_size = 1000
X_train, X_test, y_train, y_test = train_test_split(X[0:100000], y[0:100000], train_size=0.5, random_state=0)
x_target_train = np.array(X_train)[:tr_size]
y_target_train = np.array(y_train)[:tr_size]
x_target_test = np.array(X_test)[:tr_size]
y_target_test = np.array(y_test)[:tr_size]

x_shadow = X[100000:]
y_shadow = y[100000:]

In [111]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 1
model.epsilon        = 0

In [112]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

The accuracy of the model : 84.39999999999999 %
The accuracy of the model : 57.99999999999999 %


0.58

In [58]:
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)

The accuracy of the model : 88.0 %
The accuracy of the model : 83.7 %


0.8371

In [144]:
s_ms = {}
number_of_sms = 10
shadow_size = 40000
shadow_batch_size = int(shadow_size/number_of_sms)

x_shadow_train = np.array(x_shadow[:shadow_size])
y_shadow_train = np.array(y_shadow[:shadow_size])
x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

for i in range(number_of_sms):
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    shadow_model = algo.LogisticRegression_DPSGD()

    shadow_model.n_classes      = len(np.unique(y_target_test))
    shadow_model.alpha          = 0.001
    shadow_model.max_iter       = 100
    shadow_model.lambda_        = 1e-5
    shadow_model.tolerance      = 1e-5
    shadow_model.DP             = False

    X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
    shadow_model.SGD(X,y)
    print('Shadow model: ', i)
    shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end], acc=True)
    shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end], acc=True)
    s_ms[i] = shadow_model

/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_4428/3992013641.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_train = np.array(y_shadow[:shadow_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_4428/3992013641.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])


Shadow model:  0
The accuracy of the model : 86.2 %
The accuracy of the model : 78.4 %
Shadow model:  1
The accuracy of the model : 87.0 %
The accuracy of the model : 78.3 %
Shadow model:  2
The accuracy of the model : 86.7 %
The accuracy of the model : 78.9 %
Shadow model:  3
The accuracy of the model : 87.1 %
The accuracy of the model : 80.30000000000001 %
Shadow model:  4
The accuracy of the model : 86.3 %
The accuracy of the model : 79.3 %
Shadow model:  5
The accuracy of the model : 87.0 %
The accuracy of the model : 79.7 %
Shadow model:  6
The accuracy of the model : 85.6 %
The accuracy of the model : 78.8 %
Shadow model:  7
The accuracy of the model : 86.6 %
The accuracy of the model : 78.8 %
Shadow model:  8
The accuracy of the model : 86.3 %
The accuracy of the model : 78.4 %
Shadow model:  9
The accuracy of the model : 86.4 %
The accuracy of the model : 78.2 %


In [145]:
shadow_train_pred = []
shadow_test_pred = []

for i in range(number_of_sms): 
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
    shadow_train_pred.append(train_prediciton)
    shadow_test_pred.append(test_prediciton)
    
    

In [146]:
y_shadow_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_train.reshape(-1,1)) #encoode the target values
y_shadow_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_shadow_test.reshape(-1,1)) #encoode the target values


In [147]:
shadow_train_pred[0].shape

(4000, 7)

In [148]:
sh_train_pred = np.concatenate(shadow_train_pred)
sh_test_pred = np.concatenate(shadow_test_pred)

# members
labels = np.ones(sh_train_pred.shape[0])
# non-members
test_labels = np.zeros(sh_test_pred.shape[0])

x_1 = np.concatenate((sh_train_pred, sh_test_pred))
x_2 = np.concatenate((y_shadow_train_ohe, y_shadow_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))


In [149]:
attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)

In [150]:
k = 3000


In [151]:

attack_params = model_params('attack_model')
attack_params.learning_rate = 0.001
attack_params.weight_decay = 1e-5
attack_params.epoch = 200
attack_params.h_neurons=64
attack_params.do=0

for attack_params.learning_rate in [0.001, 0.01]:
    for attack_params.h_neurons in [32,64,128]:
        for attack_params.do in [0, 0.5]:
            # attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
            # attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
            attack_train_args = Train_args(attack_params.learning_rate, attack_params.weight_decay, attack_params.epoch)
            attack_model = Net_attack(attack_params.h_neurons, attack_params.do, input_size=attack_train_data.shape[1])
            for epoch in range(attack_train_args.epoch):
                attack_model = attack.train_attack_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
                print(f'epoch {epoch}: {attack_evaluation(attack_model, attack_train_data, attack_train_target)}' )        
                
                train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)
                test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

                attack_params.number_of_sms = number_of_sms
                attack_params.shadow_size = shadow_size

                attack_params.train_acc, attack_params.train_pre, attack_params.train_rec = train_res
                attack_params.test_acc, attack_params.test_pre, attack_params.test_rec = test_res

                a_param = dict(shadow_model.__dict__)
                a_param.pop('theta')
                a_param.pop('pred_func')
                a_param.pop('accuracy')
                a_param.update(dict(attack_params.__dict__))
                a_param

                k+=1
                at_path = 'mia/loan/attack_model'+str(k)
                torch.save(attack_model, at_path)
                with open(at_path+'_params.json', 'w') as file:
                    json.dump(a_param, file)


epoch 0: (0.491625, 0.4947346913114548, 0.786925)
epoch 1: (0.4910625, 0.4940220390281424, 0.7386)
epoch 2: (0.4902875, 0.49315165083114454, 0.6994)
epoch 3: (0.4905875, 0.493071784774488, 0.669875)
epoch 4: (0.4907625, 0.4930218503899832, 0.65265)
epoch 5: (0.4914625, 0.4933772907980219, 0.636025)
epoch 6: (0.4928, 0.4942602040816326, 0.62)
epoch 7: (0.4937875, 0.49488798831540187, 0.601425)
epoch 8: (0.4942125, 0.4951407401188052, 0.589725)
epoch 9: (0.4956, 0.4962322315464977, 0.5795)
epoch 10: (0.497325, 0.49766528474798166, 0.5702)
epoch 11: (0.4990625, 0.4991692328141962, 0.5633)
epoch 12: (0.5003, 0.5002689256420599, 0.558075)
epoch 13: (0.5011375, 0.5010283647869815, 0.5542)
epoch 14: (0.501825, 0.5016619615699845, 0.550875)
epoch 15: (0.5025625, 0.5023502166785133, 0.547725)
epoch 16: (0.503075, 0.5028378939596696, 0.54485)
epoch 17: (0.5035625, 0.5033034286111691, 0.542775)
epoch 18: (0.50425, 0.5039597503028045, 0.5409)
epoch 19: (0.5049, 0.5045852243484771, 0.539225)
epoch 

epoch 164: (0.545425, 0.5372687369241498, 0.65485)
epoch 165: (0.54525, 0.5360902855319828, 0.67215)
epoch 166: (0.54545, 0.5372403621614978, 0.655675)
epoch 167: (0.545525, 0.5363255535607421, 0.67215)
epoch 168: (0.5454875, 0.5372657450076805, 0.6558)
epoch 169: (0.5456, 0.5364042790994731, 0.6719)
epoch 170: (0.5456, 0.537337263571604, 0.65625)
epoch 171: (0.545775, 0.5364799171182658, 0.673175)
epoch 172: (0.545725, 0.5374365482233503, 0.656425)
epoch 173: (0.5457875, 0.5364339851598401, 0.67415)
epoch 174: (0.545925, 0.5375112309074573, 0.658075)
epoch 175: (0.5458125, 0.5365454799274076, 0.6726)
epoch 176: (0.5457875, 0.5374271993460713, 0.657475)
epoch 177: (0.545775, 0.5363842301883793, 0.674825)
epoch 178: (0.5455875, 0.537142275180772, 0.659275)
epoch 179: (0.5458, 0.5365420672597438, 0.672475)
epoch 180: (0.54575, 0.5373759241861035, 0.657775)
epoch 181: (0.5460125, 0.5365824571167339, 0.6749)
epoch 182: (0.54565, 0.5371909242739011, 0.659375)
epoch 183: (0.5456875, 0.536470

epoch 131: (0.5426875, 0.5343499165141121, 0.66405)
epoch 132: (0.5417, 0.5355332111967961, 0.628475)
epoch 133: (0.5423125, 0.5346192395017284, 0.653425)
epoch 134: (0.542625, 0.5347859795160567, 0.6553)
epoch 135: (0.5426125, 0.5349032456230163, 0.65305)
epoch 136: (0.5412875, 0.5356272246791068, 0.620725)
epoch 137: (0.5423875, 0.5352487474272885, 0.64365)
epoch 138: (0.5424125, 0.5351467815782386, 0.645775)
epoch 139: (0.5415625, 0.5355592154514148, 0.625975)
epoch 140: (0.5426875, 0.5345291298457059, 0.660825)
epoch 141: (0.5430625, 0.53423024184734, 0.672075)
epoch 142: (0.542275, 0.5354968722448465, 0.63775)
epoch 143: (0.5431125, 0.534432841483138, 0.66915)
epoch 144: (0.5427875, 0.5346128177644751, 0.660875)
epoch 145: (0.543225, 0.5343628269337786, 0.672175)
epoch 146: (0.542225, 0.5355130361648444, 0.636725)
epoch 147: (0.5427625, 0.534301241301863, 0.6661)
epoch 148: (0.54395, 0.5341332712022367, 0.68775)
epoch 149: (0.5424875, 0.5347965848365103, 0.653)
epoch 150: (0.54327

epoch 95: (0.5435375, 0.5340369393139842, 0.6831)
epoch 96: (0.5427625, 0.5354382911720222, 0.6461)
epoch 97: (0.543475, 0.5340140046160466, 0.68255)
epoch 98: (0.5427875, 0.5354443224884544, 0.646375)
epoch 99: (0.5436375, 0.5341524976031619, 0.6825)
epoch 100: (0.5427875, 0.5354369836635816, 0.6465)
epoch 101: (0.54375, 0.5342559605371334, 0.682325)
epoch 102: (0.542975, 0.5355871149387215, 0.646775)
epoch 103: (0.5439, 0.5343989970224102, 0.682)
epoch 104: (0.543325, 0.5358710051333002, 0.647225)
epoch 105: (0.544075, 0.5345821890937622, 0.681325)
epoch 106: (0.5434125, 0.5359620601818295, 0.647)
epoch 107: (0.5441875, 0.5347092669324274, 0.680725)
epoch 108: (0.5431625, 0.5358024179333513, 0.64595)
epoch 109: (0.544375, 0.5349946768660542, 0.6784)
epoch 110: (0.543125, 0.5358032378580324, 0.645375)
epoch 111: (0.544525, 0.5350742447516641, 0.67925)
epoch 112: (0.543325, 0.5360365980453317, 0.64445)
epoch 113: (0.544675, 0.5351329034287512, 0.680475)
epoch 114: (0.5434625, 0.5360916

epoch 57: (0.5375875, 0.5333244674956225, 0.60155)
epoch 58: (0.5386, 0.5330408731007917, 0.622725)
epoch 59: (0.5385125, 0.5341778892019613, 0.601925)
epoch 60: (0.5373125, 0.5337036786125601, 0.59085)
epoch 61: (0.5408625, 0.5335550491675392, 0.64975)
epoch 62: (0.5381875, 0.5349374442487592, 0.5847)
epoch 63: (0.5384375, 0.534100738572094, 0.602025)
epoch 64: (0.5399625, 0.5352053738575047, 0.607525)
epoch 65: (0.5417375, 0.534192147787085, 0.652075)
epoch 66: (0.5408875, 0.5356232711114983, 0.614775)
epoch 67: (0.540825, 0.5353907502925751, 0.6176)
epoch 68: (0.5413625, 0.5355752897413293, 0.6227)
epoch 69: (0.539025, 0.5360808062130178, 0.579825)
epoch 70: (0.5423125, 0.5336568099111102, 0.6709)
epoch 71: (0.5423375, 0.5348005671660193, 0.650625)
epoch 72: (0.5411625, 0.5357011210130315, 0.61765)
epoch 73: (0.5419375, 0.5342675627642841, 0.65385)
epoch 74: (0.5432875, 0.5341162098792189, 0.6777)
epoch 75: (0.541575, 0.5351942774908999, 0.632225)
epoch 76: (0.5422125, 0.53490583590

...

epoch 33: (0.5443125, 0.528334159246767, 0.826275)
epoch 34: (0.518225, 0.6021008403361344, 0.107475)
epoch 35: (0.5442, 0.5283842794759825, 0.8228)
epoch 36: (0.5187125, 0.6012170385395538, 0.11115)
epoch 37: (0.5441875, 0.5282985638578908, 0.824925)
epoch 38: (0.5189, 0.599290780141844, 0.114075)
epoch 39: (0.544275, 0.528411460839991, 0.82345)
epoch 40: (0.5188625, 0.5975184179914695, 0.115575)
epoch 41: (0.5442875, 0.5283762354034183, 0.82465)
epoch 42: (0.5191625, 0.5975563192058038, 0.117375)
epoch 43: (0.5442125, 0.5283172946055433, 0.824875)
epoch 44: (0.5194875, 0.5971581702605011, 0.119775)
epoch 45: (0.5441625, 0.5282952379426887, 0.82455)
epoch 46: (0.519675, 0.5974251052240653, 0.12065)
epoch 47: (0.544275, 0.5283795910518556, 0.824325)
epoch 48: (0.51975, 0.597844934357196, 0.120675)
epoch 49: (0.5445, 0.5285402770651616, 0.8241)
epoch 50: (0.519525, 0.5963008631319359, 0.1209)
epoch 51: (0.5443875, 0.5284941663590699, 0.823275)
epoch 52: (0.5195875, 0.596264897407544, 0.

epoch 195: (0.54505, 0.5292171995589857, 0.816)
epoch 196: (0.5196625, 0.595229446664245, 0.1229)
epoch 197: (0.54505, 0.5292171995589857, 0.816)
epoch 198: (0.5197, 0.5953764221738078, 0.122975)
epoch 199: (0.54505, 0.5292181470311639, 0.815975)
epoch 0: (0.5107125, 0.5204290822407628, 0.2729)
epoch 1: (0.5001, 0.5000500125031258, 0.99985)
epoch 2: (0.513, 0.534805890227577, 0.19975)
epoch 3: (0.5005625, 0.5002814505335051, 0.99985)
epoch 4: (0.5131125, 0.5447563785305913, 0.1596)
epoch 5: (0.5034875, 0.501751478398433, 0.999075)
epoch 6: (0.5150625, 0.5653399848172649, 0.130325)
epoch 7: (0.5082375, 0.5041784495986406, 0.99395)
epoch 8: (0.5144375, 0.5797321551843159, 0.104975)
epoch 9: (0.511125, 0.5057041043915195, 0.9863)
epoch 10: (0.5125, 0.5899280575539568, 0.082)
epoch 11: (0.512425, 0.5064107525217346, 0.9815)
epoch 12: (0.5117375, 0.5967841682127396, 0.072375)
epoch 13: (0.5149375, 0.5077851177214036, 0.9743)
epoch 14: (0.5106875, 0.6024442846872753, 0.06285)
epoch 15: (0.51

epoch 159: (0.54235, 0.5257830811847433, 0.863625)
epoch 160: (0.5149375, 0.6389858106536404, 0.068675)
epoch 161: (0.54245, 0.525851044394373, 0.8635)
epoch 162: (0.5148, 0.6378667908709827, 0.068475)
epoch 163: (0.542275, 0.5257609457359618, 0.8628)
epoch 164: (0.51485, 0.6375636868920797, 0.068825)
epoch 165: (0.542375, 0.5258140172398038, 0.86315)
epoch 166: (0.51495, 0.6381062355658199, 0.069075)
epoch 167: (0.5423875, 0.5258235983977336, 0.8631)
epoch 168: (0.5149125, 0.6373474556758001, 0.0692)
epoch 169: (0.5424375, 0.5258422518915463, 0.863525)
epoch 170: (0.5148375, 0.6367826688177, 0.069075)
epoch 171: (0.5425, 0.5258948971820259, 0.863125)
epoch 172: (0.5148625, 0.6370763200368919, 0.069075)
epoch 173: (0.5425875, 0.525920177720972, 0.8641)
epoch 174: (0.515, 0.6377410468319559, 0.06945)
epoch 175: (0.5425875, 0.5259312255491453, 0.86375)
epoch 176: (0.5149625, 0.6374913852515507, 0.069375)
epoch 177: (0.5426, 0.5259479214253083, 0.863475)
epoch 178: (0.5160125, 0.628023186

epoch 123: (0.5412625, 0.5243117441706314, 0.889875)
epoch 124: (0.5100125, 0.6687737041719343, 0.039675)
epoch 125: (0.541275, 0.5243187509206069, 0.8899)
epoch 126: (0.5100125, 0.6687737041719343, 0.039675)
epoch 127: (0.541275, 0.5243201838376101, 0.88985)
epoch 128: (0.5100125, 0.6687737041719343, 0.039675)
epoch 129: (0.5412375, 0.5242970142438392, 0.88985)
epoch 130: (0.5100125, 0.6687737041719343, 0.039675)
epoch 131: (0.54125, 0.5243054532598769, 0.889825)
epoch 132: (0.5100125, 0.6687737041719343, 0.039675)
epoch 133: (0.54125, 0.5243054532598769, 0.889825)
epoch 134: (0.5100375, 0.6690526315789473, 0.039725)
epoch 135: (0.54125, 0.5243061693477108, 0.8898)
epoch 136: (0.5100125, 0.6686315789473685, 0.0397)
epoch 137: (0.54125, 0.5243068854777407, 0.889775)
epoch 138: (0.510025, 0.6687710437710438, 0.039725)
epoch 139: (0.5412625, 0.5243160422528986, 0.889725)
epoch 140: (0.510025, 0.6687710437710438, 0.039725)
epoch 141: (0.5412875, 0.5243322086838654, 0.8897)
epoch 142: (0.5

epoch 87: (0.538975, 0.5225883102958648, 0.9017)
epoch 88: (0.510275, 0.6689967105263158, 0.040675)
epoch 89: (0.53895, 0.5225764381973627, 0.901575)
epoch 90: (0.510275, 0.6684426229508197, 0.040775)
epoch 91: (0.5388375, 0.5225448793301116, 0.900175)
epoch 92: (0.5102625, 0.66872174270448, 0.040675)
epoch 93: (0.5388125, 0.5225094604978904, 0.90095)
epoch 94: (0.510275, 0.6674816625916871, 0.04095)
epoch 95: (0.5390125, 0.5226846535157216, 0.8989)
epoch 96: (0.5102875, 0.6683026584867076, 0.04085)
epoch 97: (0.539, 0.5226684878956087, 0.899225)
epoch 98: (0.51045, 0.668141592920354, 0.041525)
epoch 99: (0.539175, 0.5227616059496834, 0.899725)
epoch 100: (0.5104375, 0.6678729392842783, 0.041525)
epoch 101: (0.5392375, 0.5228188017039589, 0.899)
epoch 102: (0.5104375, 0.6677380474085979, 0.04155)
epoch 103: (0.539325, 0.522887323943662, 0.898425)
epoch 104: (0.510425, 0.6673354735152488, 0.041575)
epoch 105: (0.539375, 0.5229177579884756, 0.898425)
epoch 106: (0.5104375, 0.667468912956

In [130]:
train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)
test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

attack_params.number_of_sms = number_of_sms
attack_params.shadow_size = shadow_size

attack_params.train_acc, attack_params.train_pre, attack_params.train_rec = train_res
attack_params.test_acc, attack_params.test_pre, attack_params.test_rec = test_res

a_param = dict(shadow_model.__dict__)
a_param.pop('theta')
a_param.pop('pred_func')
a_param.pop('accuracy')
a_param.update(dict(attack_params.__dict__))
a_param

k+=1
at_path = 'mia/loan/attack_model'+str(k)
torch.save(attack_model, at_path)
with open(at_path+'_params.json', 'w') as file:
    json.dump(a_param, file)
        

{'n_classes': 7,
 'alpha': 0.001,
 'max_iter': 100,
 'lambda_': 1e-05,
 'tolerance': 1e-05,
 'sgdDP': False,
 'L': 1,
 'C': 1,
 'epsilon': 1,
 'delta': 1e-05,
 'sigma': 0,
 'DP': False,
 'model_name': 'attack_model',
 'learning_rate': 0.001,
 'weight_decay': 1e-05,
 'epoch': 200,
 'h_neurons': 64,
 'do': 0,
 'number_of_sms': 10,
 'shadow_size': 20000,
 'train_acc': 0.58185,
 'train_pre': 0.5673219279486758,
 'train_rec': 0.68975,
 'test_acc': 0.6395,
 'test_pre': 0.6368989205103042,
 'test_rec': 0.649}

In [61]:
sh_path = f'mia/shms{number_of_sms}_shtrsize{shadow_size}_shlr{shadow_model.alpha}_shiter{int(shadow_model.max_iter/shadow_batch_size)}_shreg{shadow_model.lambda_}/'      
os.mkdir(sh_path)

torch.save(attack_train_data, sh_path+'attack_train_data.pt')
torch.save(attack_train_target, sh_path+'attack_train_target.pt')

at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
torch.save(attack_model, at_path)


In [74]:
train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
train_acc, train_pre, train_rec 

(0.59217, 0.5930014328093153, 0.5877)

In [75]:
test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

(0.6260389215487533, 0.5935737236701178, 0.8313)

In [ ]:
# MI optimization

In [ ]:
ams

In [87]:
rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

epo = [200]
wd = [1e-5, 1e-6]
ams = {}
for ep in epo:
    for w_d in wd:
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=ep)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        for epoch in range(attack_train_args.epoch):

            attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
            ams[(l_r,w_d)] = attack_model
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        torch.save(attack_model, at_path)


In [91]:
epo = [200,500]
wd = [1e-5, 1e-6]

for ep in epo:
    for w_d in wd:
        print(ep, w_d)
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=500)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        attack_model = torch.load(at_path)
        
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print('Train: ', train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print('Test: ' , test_acc, test_pre, test_rec)


200 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
200 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99
500 1e-05
Train:  0.8081 0.8207236842105263 0.78842
Test:  0.6895398337725522 0.6209717711716213 0.9943
500 1e-06
Train:  0.80328 0.8273606493674712 0.7665
Test:  0.7008919521589296 0.6304929308368361 0.99


In [ ]:
for l_r in lr:
    for w_d in wd:
        ams[(l_r,w_d)]
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print("Train acc pre rec: ", train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print("Test acc pre rec: ", test_acc, test_pre, test_rec)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
shadow_size = 20000
shadow_clf = LogisticRegression(random_state=1).fit(x_shadow[:shadow_size], y_shadow[:shadow_size])

In [ ]:
print(shadow_clf.score(x_shadow[:shadow_size], y_shadow[:shadow_size]))
print(shadow_clf.score(x_shadow[shadow_size:shadow_size*2], y_shadow[shadow_size:shadow_size*2]))

In [ ]:
shadow_train_pred = shadow_clf.predict_proba(x_shadow[:shadow_size])
shadow_test_pred = shadow_clf.predict_proba(x_shadow[shadow_size:shadow_size*2])
y_shadow_train = y_shadow[:shadow_size]
y_shadow_test = y_shadow[shadow_size:shadow_size*2]

In [ ]:
# members
labels = np.ones(shadow_train_pred.shape[0])
# non-members
test_labels = np.zeros(shadow_test_pred.shape[0])

x_1 = np.concatenate((shadow_train_pred, shadow_test_pred))
x_2 = np.concatenate((y_shadow_train, y_shadow_test)).reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = np.array(df.drop(['a_target'], axis=1))
attack_train_target = np.array(df['a_target'])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [ ]:
train_acc, train_pre, train_rec 

In [67]:
train_pred = model.predict(x_target_train, y_target_train)
test_pred = model.predict(x_target_test, y_target_test)

y_target_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_train.reshape(-1,1)) #encoode the target values
y_target_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_test.reshape(-1,1)) #encoode the target values
    
# members
labels = np.ones(train_pred.shape[0])
# non-members
test_labels = np.zeros(test_pred.shape[0])

x_1 = np.concatenate((train_pred, test_pred))
x_2 = np.concatenate((y_target_train_ohe, y_target_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_test_data = np.concatenate((x_1,x_2),axis=1)
attack_test_target = y_new
df = pd.DataFrame(attack_test_data)
df['a_target'] = attack_test_target
df = df.sample(frac = 1)

attack_test_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_test_target = torch.tensor(np.array(df['a_target']))


In [ ]:
test_acc, test_pre, test_rec = attack_evaluation(a_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

In [14]:
a_model = Net_attack(h_neurons=64, do=0, input_size=100)
a_model = torch.load('attack_model') 

In [ ]:

rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

lr = [0.001]
wd = [1e-4, 1e-6]
tms = {}
for l_r in lr:
    for w_d in wd:
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = 100
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size 
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.epsilon

        X,y = model.init_theta(x_target_train, y_target_train)
        model.SGD(X,y)
        model.evaluate(x_target_train, y_target_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import algo
# import attack

# from torch import nn,optim
import torch

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
        

raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")

for L in [1,10,100]:
    for epsilon in np.arange(0,1,0.1):
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = n_classes
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.L              = L
        model.epsilon        = epsilon


        X,y = model.init_theta(x_target_train, y_target_train)
        model.train(X,y)
        model.evaluate(x_target_train, y_target_train, acc=True)
        model.evaluate(x_target_test, y_target_test, acc=True)

        tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
        if model.DP:
            tm_path += f'_eps{model.epsilon}_L{model.L}'
        np.save(tm_path+'_target_model', model.theta)

        print(tm_path)

    

In [ ]:
#Shadow models
# s_ms = {}
# number_of_sms = 10
# shadow_size = 50000
# shadow_batch_size = int(shadow_size/number_of_sms)

# x_shadow_train = np.array(x_shadow[:shadow_size])
# y_shadow_train = np.array(y_shadow[:shadow_size])
# x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
# y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

# attack.train_shadow_models(number_of_sms,)

# for i in range(number_of_sms):  
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     shadow_model = algo.LogisticRegression_DPSGD()

#     shadow_model.n_classes      = n_classes
#     shadow_model.alpha          = 0.001
#     shadow_model.max_iter       = 100*shadow_batch_size
#     shadow_model.lambda_        = 10e-3
#     shadow_model.tolerance      = 10e-5
#     shadow_model.DP             = False

#     X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
#     shadow_model.SGD(X,y)
#     print('Shadow model: ', i)
#     shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
#     s_ms[i] = shadow_model

# #Attack model

# shadow_train_pred = []
# shadow_test_pred = []

# for i in range(number_of_sms): 
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
#     shadow_train_pred.append(train_prediciton)
#     shadow_test_pred.append(test_prediciton)
    

In [10]:
1/10000 == 1e-5


False

In [3]:
np.arange(0,1,0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])